## Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Portanto, você deve implementar o modelo de linguagem inspirado no artigo do Bengio, para prever a próxima palavra usando rede com embeddings e duas camadas.
Sugestão de alguns parâmetros:
* context_size = 9
* max_vocab_size = 3000
* embedding_dim = 64
* usar pontuação no vocabulário
* descartar qualquer contexto ou target que não esteja no vocabulário
* É esperado conseguir uma perplexidade da ordem de 50.
* Procurem fazer asserts para garantir que partes do seu programa estão testadas

Este enunciado não é fixo, podem mudar qualquer um dos parâmetros acima, mas procurem conseguir a perplexidade esperada ou menor.

Gerem alguns frases usando um contexto inicial e depois deslocando o contexto e prevendo a próxima palavra gerando frases compridas para ver se está gerando texto plausível.

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída).
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de device. Somente ligue a device quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

Procure por `TODO` para entender onde você precisa inserir o seu código.

## Faz download e carrega o dataset

In [1]:
!wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
!wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [2]:
# Simples limitação dos dados, para trabalhar apenas com tokens presentes no livro.

text = open("67724.txt.utf-8","r",encoding="utf-8").read()
idx = text.find("PARTE\n\n")
idx2 = text.find("*** END OF THE PROJECT")
text = text[idx:idx2]
text2 = open("67725.txt.utf-8","r",encoding="utf-8").read()
idx = text2.find("PARTE\n\n")
idx2 = text2.find("*** END OF THE PROJECT")
text2 = text2[idx:idx2]

text += text2

paragraphs = text.split("\n\n")
len(paragraphs)

4816

In [3]:
# cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]
import re

cleaned_paragraphs = []
full_text = ""
final_tokens = []
# Tratando tokens em cada prágrafo
for paragraph in paragraphs:
    paragraph = paragraph.replace("\n", " ")
    for removable in ["«", "»", "_"]:
        paragraph = paragraph.replace(removable, '') # Removendo as aspas, underline, etc.
    
    paragraph = paragraph.lower().strip() # Caixa baixa e removendo leading e trailing spaces.

    if paragraph[:3] == "pag":
        continue
    if len(paragraph) < 3:
        continue

    paragraph = re.sub("[ ]+", " ", paragraph) # Espaços duplicados

    for punctuation in ['.', ',', ';', '!', ":", "?", "--"]:
        paragraph = paragraph.replace(punctuation, (' ' + punctuation) if punctuation != "--" else (punctuation + ' ')) # Tratando pontuação como próprio token
    cleaned_paragraphs.append(paragraph)
    final_tokens += paragraph.split(" ") + ['\n']
    full_text += paragraph + '\n'
    
for paragraph in cleaned_paragraphs:
    print(paragraph)

# print(final_tokens)

parte
os aventureiros
scenario
de um dos cabeços da serra dos órgãos deslisa um fio d'agua que se dirige para norte , e engrossado com os mananciaes , que recebe no seu curso de dez leguas , torna-se rio caudal .
é o paquequer : soltando de cascata em cascata , enroscando-se como uma serpente , vai depois se espreguiçar na varzea e embeber no parahyba , que rola magestosamente em seu vasto leito .
dir-se-hia que vassallo e tributario desse rei das aguas , o pequeno rio , altivo e sobranceiro contra os rochedos , curva-se humildemente aos pés do suzerano . perde então a belleza selvatica ; suas ondas são calmas e serenas como as de um lago , e não se revoltão contra os barcos e as canôas que resvalão sobre ellas : escravo submisso , soffre o latego do senhor .
não é neste lugar que elle deve ser visto ; sim tres ou quatro leguas acima de sua foz , onde é livre ainda , como o filho indomito desta patria da liberdade .
ahi , o paquequer lança-se rapido sobre o seu leito , e atravessa as f

In [4]:
final_tokens

['parte',
 '\n',
 'os',
 'aventureiros',
 '\n',
 'scenario',
 '\n',
 'de',
 'um',
 'dos',
 'cabeços',
 'da',
 'serra',
 'dos',
 'órgãos',
 'deslisa',
 'um',
 'fio',
 "d'agua",
 'que',
 'se',
 'dirige',
 'para',
 'norte',
 ',',
 'e',
 'engrossado',
 'com',
 'os',
 'mananciaes',
 ',',
 'que',
 'recebe',
 'no',
 'seu',
 'curso',
 'de',
 'dez',
 'leguas',
 ',',
 'torna-se',
 'rio',
 'caudal',
 '.',
 '\n',
 'é',
 'o',
 'paquequer',
 ':',
 'soltando',
 'de',
 'cascata',
 'em',
 'cascata',
 ',',
 'enroscando-se',
 'como',
 'uma',
 'serpente',
 ',',
 'vai',
 'depois',
 'se',
 'espreguiçar',
 'na',
 'varzea',
 'e',
 'embeber',
 'no',
 'parahyba',
 ',',
 'que',
 'rola',
 'magestosamente',
 'em',
 'seu',
 'vasto',
 'leito',
 '.',
 '\n',
 'dir-se-hia',
 'que',
 'vassallo',
 'e',
 'tributario',
 'desse',
 'rei',
 'das',
 'aguas',
 ',',
 'o',
 'pequeno',
 'rio',
 ',',
 'altivo',
 'e',
 'sobranceiro',
 'contra',
 'os',
 'rochedos',
 ',',
 'curva-se',
 'humildemente',
 'aos',
 'pés',
 'do',
 'suzerano

In [5]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        if text == "\n":
            word_counts.update(text)
            continue
        # word_counts.update(re.findall(r'\w+', text.lower()))
        word_counts.update(list(re.findall(r'.*', text.lower())))
        
    return word_counts

word_counts = count_words(final_tokens)
word_counts.pop('')

print(word_counts)

Counter({',': 7377, '.': 5956, '\n': 4660, 'a': 4378, 'que': 4335, 'o': 3952, 'de': 3942, 'e': 3605, ';': 2357, '--': 1733, 'um': 1657, 'do': 1400, 'não': 1279, 'uma': 1211, 'da': 1131, 'se': 1093, 'os': 1088, 'com': 1015, 'sua': 925, 'para': 854, 'seu': 777, '!': 770, 'pery': 729, 'em': 721, 'as': 691, 'no': 642, '?': 627, 'por': 621, 'como': 594, 'ao': 592, 'á': 490, 'era': 469, 'cecilia': 457, 'na': 452, 'tinha': 451, 'é': 441, ':': 430, 'd': 423, 'sobre': 416, 'mas': 410, 'elle': 404, 'dos': 371, 'indio': 339, 'seus': 324, 'mais': 316, 'antonio': 302, 'quando': 287, 'alvaro': 278, 'lhe': 272, 'das': 258, 'disse': 243, 'ella': 231, 'senhora': 228, 'olhos': 223, 'menina': 215, 'pela': 213, 'tu': 204, 'me': 200, 'depois': 198, 'isabel': 197, 'havia': 194, 'fidalgo': 194, 'casa': 192, 'estava': 187, 'ainda': 185, 'tempo': 181, 'já': 181, 'mariz': 178, 'nos': 175, 'momento': 174, 'loredano': 174, 'aventureiros': 173, 'só': 173, 'mesmo': 173, 'italiano': 173, 'todos': 170, 'pelo': 169, '

## Criando um vocabulário

In [6]:
vocab_size = 2500
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [7]:
def encode_sentence(sentence, vocab):
    if isinstance(sentence, str):
        sentence = sentence.split(" ")
    # print(sentence)
    return [vocab.get(word, 0) for word in sentence]

encode_sentence(cleaned_paragraphs[20], vocab)

[1360,
 2386,
 50,
 886,
 1243,
 1,
 1536,
 225,
 0,
 1,
 11,
 0,
 7,
 0,
 1,
 11,
 1120,
 879,
 1,
 0,
 11,
 103,
 8,
 1366,
 14,
 335,
 1357,
 86,
 104,
 4,
 91,
 12,
 82,
 35,
 0,
 26,
 0,
 593,
 18,
 14,
 1362,
 8,
 580,
 945,
 2]

## Classe do dataset

In [8]:
context_size = 5 # 5 palavras de entrada. O target é a próxima palavra
"""TODO: Preparar o dataset"""
overlap_size = 4
step = context_size - overlap_size
if step <= 0:
    raise

# print(final_tokens)
whole_data = []
for i in range(0, len(final_tokens) - context_size, step):
    cur_data = [encode_sentence(final_tokens[i:i+context_size], vocab), encode_sentence(final_tokens[i + context_size], vocab)[0]]
    if 0 in cur_data[0] or 0 == cur_data[1]:# or vocab_size in cur_data[0] or vocab_size == cur_data[1] :
        continue
    for i in range(5):
        cur_data[0][i] -= 1
    cur_data[1] -= 1
    whole_data.append(tuple(cur_data))

print(whole_data[:5])

[([1, 2, 35, 5, 591], 36), ([2, 35, 5, 591, 36], 1355), ([35, 5, 591, 36, 1355], 6), ([5, 591, 36, 1355, 6], 1356), ([591, 36, 1355, 6, 1356], 23)]


In [9]:
"""TODO: divida o dataset em validação/treino com um proporção de 20/80 %. OBS, use random_state=18"""
import numpy as np

N = len(whole_data)
random_state = 18
np.random.seed(random_state)
random_indices = np.arange(N)
np.random.shuffle(random_indices)
# print(random_indices)
cut_idx = int(0.8 * N)
train_indices = random_indices[:cut_idx]
validation_indices = random_indices[cut_idx:]

In [10]:
"""TODO: implemente a classe do dataset"""
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, split, vocab):
        idxs = train_indices if split == "train" else validation_indices
        self.data = []
        for idx in idxs:
            self.data.append(whole_data[idx])
            
        self.vocab = vocab  # Set vocabulary

    def __len__(self):
        return len(self.data)  # Return the length of the dataset

    def __getitem__(self, idx):
        line, label = self.data[idx]  # Get label and text for specified index

        return torch.tensor(line), torch.tensor(label)

train_data = MyDataset(split="train", vocab=vocab)
val_data = MyDataset(split="val", vocab=vocab)

In [11]:
batch_size = 30
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
sample = next(iter(train_loader))
print(sample)

[tensor([[   1,    2,    5,   42,  222],
        [  98, 1628,   26,  425, 1852],
        [   1,    2,  112,    0,    3],
        [ 182,    0,   17,   15,    3],
        [1424,    4, 1603,   24,  224],
        [   4,   88,  779,   33, 2376],
        [   8,    3,  526,  617,    4],
        [   3,  124,    1,    2,   46],
        [1775,    5,    4, 1109,  441],
        [ 125,    1,    5,  128,    6],
        [  62,   78,   18,  320,    1],
        [   5,  145,   19,   12,  147],
        [1133,    1,    2,   22,    0],
        [  52,   66,   12,  593,  282],
        [1389,    0, 1047, 1251,    0],
        [   1,    2,    9,  161,   26],
        [  17,   51,   23,  270,    6],
        [   0,  231, 1479,    0,  226],
        [   5,    4,   35,   26,    2],
        [   8,    5,   61,    4,   15],
        [  56,  269,   10,  149,    0],
        [   4,  399,   14,  124,    1],
        [   1,    2,   47,    7,   16],
        [   6, 1917,    3, 1040,    1],
        [  36,    2,    9,    5,    4],

## Model

In [12]:
import torch.nn as nn
import torch
import multiprocessing
import torch.nn.functional as F
import torch.optim as optim
import time

class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(LanguageModel, self).__init__()
        self.context_size = context_size
        self.embedding_dim = embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.linear2 = nn.Linear(h, h) # This hidden layer ideia I've got from Gabriel Freita's code. It helped to reduce PPL in 20.
        self.linear3 = nn.Linear(h, vocab_size, bias = False)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = embeds.view(embeds.size(0), -1)
        out = torch.tanh(self.linear1(embeds))
        out = self.relu(self.linear2(out))
        out = self.linear3(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

embedding_dim = 128
context_size = 5
H = 500
model = LanguageModel(vocab_size, embedding_dim, context_size, H)

In [13]:
# helper function to get accuracy from log probabilities
def get_accuracy_from_log_probs(log_probs, labels):
    probs = torch.exp(log_probs)
    predicted_label = torch.argmax(probs, dim=1)
    acc = (predicted_label == labels).float().mean()
    return acc

# helper function to evaluate model on dev data
def evaluate(model, criterion, dataloader, device):
    model.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        dev_st = time.time()
        for it, data_tensor in enumerate(dataloader):
            input = data_tensor[:,0:2]
            target = data_tensor[:,2]
            input, target = input.to(device), target.to(device)
            log_probs = model(input)
            mean_loss += criterion(log_probs, target).item()
            mean_acc += get_accuracy_from_log_probs(log_probs, target)
            count += 1
            if it % 500 == 0: 
                print(f"Dev Iteration {it} complete. Mean Loss: {mean_loss / count}; Mean Acc: {mean_acc / count}; Time taken (s): {time.time()-dev_st}")
                dev_st = time.time()

    return mean_acc / count, mean_loss / count

In [14]:
# Verifica se há uma device disponível e define o dispositivo para device se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

In [15]:
sample = next(iter(train_loader))
input = sample[0]
target = sample[1]
print(input.shape, target.shape)
output = model(input)
print(output.shape)

torch.Size([30, 5]) torch.Size([30])
torch.Size([30, 2500])


## Training and Eval

In [16]:
# Verifica se há uma device disponível e define o dispositivo para device se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
# helper function to get accuracy from log probabilities
def get_accuracy_from_log_probs(log_probs, labels):
    probs = torch.exp(log_probs)
    predicted_label = torch.argmax(probs, dim=1)
    acc = (predicted_label == labels).float().mean()
    return acc

# helper function to evaluate model on dev data
def evaluate(model, criterion, dataloader):
    model.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for context_tensor, target_tensor in dataloader:
            context_tensor, target_tensor = context_tensor.to(device), target_tensor.to(device)
            log_probs = model(context_tensor)
            mean_loss += criterion(log_probs, target_tensor).item()
            mean_acc += get_accuracy_from_log_probs(log_probs, target_tensor)
            count += 1

    return mean_acc / count, mean_loss / count

In [18]:
# Using negative log-likelihood loss
loss_function = nn.NLLLoss()

# create model
model = LanguageModel(len(vocab), embedding_dim, context_size, H)

# load it to gpu
model = model.to(device)

# optimizer = optim.Adam(model.parameters(), lr = 1e-3)
optimizer = optim.SGD(model.parameters(), lr = 1e-2)

train_acc, train_loss = evaluate(model, loss_function, train_loader)
print("\n--- Evaluating model on train data ---")
print(f"Train Accuracy: {train_acc}; Train Loss: {train_loss}, Train PPL: {torch.exp(torch.tensor(train_loss))}")

best_test_ppl = 1e9
for epoch in range(10):
    st = time.time()
    print(f"\n--- Training model Epoch: {epoch+1} ---")
    for it, data_tensor in enumerate(train_loader):       
        context_tensor = data_tensor[0]
        target_tensor = data_tensor[1]

        context_tensor, target_tensor = context_tensor.to(device), target_tensor.to(device)

        # zero out the gradients from the old instance
        model.zero_grad()
        # get log probabilities over next words
        log_probs = model(context_tensor)
        # compute loss function
        loss = loss_function(log_probs, target_tensor)
        # backward pass and update gradient
        loss.backward()
        optimizer.step()

    print(f"Finished training of Epoch {epoch +1}\n--- Evaluating model on train data ---")
    train_acc, train_loss = evaluate(model, loss_function, train_loader)
    print(f"Train Accuracy: {train_acc}; Train Loss: {train_loss}, Train PPL: {torch.exp(torch.tensor(train_loss))}")
    print("\n--- Evaluating model on test data ---")
    test_acc, test_loss = evaluate(model, loss_function, val_loader)
    print(f"Test Accuracy: {test_acc}; Test Loss: {test_loss}, Test PPL: {torch.exp(torch.tensor(test_loss))}")

    best_test_ppl = min(best_test_ppl, (torch.exp(torch.tensor(test_loss))))

print("BEST PPL:", best_test_ppl)


--- Evaluating model on train data ---
Train Accuracy: 0.001004509860649705; Train Loss: 7.809315787616837, Train PPL: 2463.444091796875

--- Training model Epoch: 1 ---
Finished training of Epoch 1
--- Evaluating model on train data ---
Train Accuracy: 0.14092037081718445; Train Loss: 5.652646943651882, Train PPL: 285.04498291015625

--- Evaluating model on test data ---
Test Accuracy: 0.1346672773361206; Test Loss: 5.738441443853355, Test PPL: 310.5799865722656

--- Training model Epoch: 2 ---
Finished training of Epoch 2
--- Evaluating model on train data ---
Train Accuracy: 0.16599996387958527; Train Loss: 5.3109000828873185, Train PPL: 202.532470703125

--- Evaluating model on test data ---
Test Accuracy: 0.15531788766384125; Test Loss: 5.445341062780094, Test PPL: 231.67630004882812

--- Training model Epoch: 3 ---
Finished training of Epoch 3
--- Evaluating model on train data ---
Train Accuracy: 0.17796409130096436; Train Loss: 5.127539942478635, Train PPL: 168.60186767578125


## Exemplo de uso

In [19]:
i = 1000
text = " ".join(final_tokens[i: i+context_size])

inv_vocab = {v-1 : k for k, v in vocab.items()}
def generate_text(model, vocab, text, max_length, context_size):
    context = encode_sentence(text, vocab)

    final_text = context
    for i in range(max_length):
        inputs = torch.tensor(context).to(device).view((1, -1))
        pred = torch.argmax(model(inputs), dim=1)
        final_text.append(pred.item())
        context = final_text[-context_size:]

    l = ([inv_vocab[t] for t in final_text])
    decoded_sentence = " ".join(l)

    print(decoded_sentence)


context = context_size
max_length= 100
generate_text(model, vocab, text, max_length, context_size)

frente 
 a que della um momento de sua senhora , e o indio , que se tinha a sua senhora . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . . . 
 -- pery ! . .
